In [ ]:
pip install transformers

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00


In [ ]:
# Loading dataset,tokenizing the data, dynamic padding the data

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# all the tensor flow objects are also keras objects

In [ ]:
#seq classification with 2 labels
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# hugging face transformers have internal/default loss function

# most models output logits i.e, data before applying softmax so that that data can be used elesewhere
# so we need to specify to the loss function that we are using logits but not actual outputs

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",# uses default adam optimizer
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

459/459 [==============================] - 155s 211ms/step - loss: 0.6904 - accuracy: 0.6219 - val_loss: 0.7347 - val_accuracy: 0.3162


In [ ]:
#learning rate scheduling with tensorflow
# Adam's default learning rate: 1e-3 (which is very high for training transformer models)
# we are going to set it as 5e-5 (20times lower than default)
# PolynomialDecay will decay the learning rate from start to 0 in specified no. of steps

In [ ]:
# much better model

In [ ]:
len(tf_train_dataset)

459

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=lr_scheduler)

In [ ]:
#reloading the model to restart the training
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
    )

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=6
    )

Epoch 1/3
459/459 [==============================] - 75s 163ms/step - loss: 0.2871 - accuracy: 0.8841 - val_loss: 0.3480 - val_accuracy: 0.8554
Epoch 3/3
459/459 [==============================] - 72s 156ms/step - loss: 0.0949 - accuracy: 0.9727 - val_loss: 0.4220 - val_accuracy: 0.8554


In [ ]:
#model predictions
preds = model.predict(tf_validation_dataset)["logits"]

51/51 [==============================] - 7s 63ms/step


In [ ]:
# we can skip softmax as it will also preserve the max value as max
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(408, 2) (408,)


In [ ]:
# metrics
# we can evaluate using keras's or
# Hugging Face 'evaluate' library

In [ ]:
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 10.1 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=class_preds, references=raw_datasets["validation"]["label"])

{'accuracy': 0.8553921568627451, 'f1': 0.8998302207130731}

In [ ]:
raw_datasets['train'].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
#fine-tune a model on the GLUE SST-2 dataset (sentiment classification)

#distilbert-base-uncased-finetuned-sst-2-english was trained on
#distilbert-base-uncased using 'sst2' dataset

# we are going to do the same

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

'LABEL_1'

In [ ]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [ ]:
inputs = tokenizer("Hello, my dog is not cute", return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'LABEL_1'

In [ ]:
#start
# download tokenzier and model for 'distilbert-base-uncased'

In [ ]:
checkpoint = 'distilbert-base-uncased'

from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_95', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
#we have the tokenizer and model
#now download the appropriate dataset : sst2
#understand how the dataset is?

In [ ]:
sst2_dataset = load_dataset('glue','sst2')

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
sst2_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [ ]:
sst2_dataset['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [ ]:
sst2_dataset['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0}

In [ ]:
# tokenize the dataset

In [ ]:
def tokenize_sample(sample):
  return tokenizer(sample['sentence'],truncation=True)

tokenized_sst2_dataset = sst2_dataset.map(tokenize_sample,batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_sst2_dataset['train'].features

{'sentence': Value(dtype='string', id=None),
 'label': ClassLabel(names=['negative', 'positive'], id=None),
 'idx': Value(dtype='int32', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [ ]:
#dynamic padding
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='tf')

In [ ]:
#train and validation datasets preparation
tf_train_dataset = tokenized_sst2_dataset['train'].to_tf_dataset(
    columns=['attention_mask','input_ids'],
    label_cols=['label'],
    shuffle=True,
    collate_fn = data_collator,
    batch_size = 8
)

tf_validation_dataset = tokenized_sst2_dataset['validation'].to_tf_dataset(
    columns=['attention_mask','input_ids'],
    label_cols=['label'],
    shuffle=False,
    collate_fn = data_collator,
    batch_size = 8
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
#adding model compilation parameters

#learning rate scheduler,optimizer and loss
from tensorflow.keras.optimizers.schedules import PolynomialDecay
epochs = 3
num_of_steps = len(tf_train_dataset) * epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,decay_steps=num_of_steps,end_learning_rate=0
)

from tensorflow.keras.optimizers import Adam
optm = Adam(learning_rate = lr_scheduler)

from tensorflow.keras.losses import SparseCategoricalCrossentropy
loss = SparseCategoricalCrossentropy(from_logits=True)

model.compile(
    optimizer = optm,
    loss = loss,
    metrics = ['accuracy']
)

In [ ]:
#training the model
model.fit(
    tf_train_dataset,
    validation_data = tf_validation_dataset,
    epochs = 3
)

Epoch 1/3


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


8419/8419 [==============================] - 492s 55ms/step - loss: 0.2212 - accuracy: 0.9136 - val_loss: 0.2725 - val_accuracy: 0.8865
Epoch 2/3
8419/8419 [==============================] - 431s 51ms/step - loss: 0.1023 - accuracy: 0.9645 - val_loss: 0.2739 - val_accuracy: 0.8888
Epoch 3/3
8419/8419 [==============================] - 428s 51ms/step - loss: 0.0505 - accuracy: 0.9826 - val_loss: 0.3163 - val_accuracy: 0.8876


In [ ]:
predictions = model.predict(tf_validation_dataset)['logits']
class_pred = np.argmax(predictions,axis=1)
class_pred.shape

109/109 [==============================] - 4s 22ms/step


(872,)

In [ ]:
#evaluation
import evaluate

metric = evaluate.load('glue','sst2')
metric.compute(predictions=class_pred, references=sst2_dataset['validation']['label'])

{'accuracy': 0.8876146788990825}

In [ ]:
#prediction of a sentence

In [ ]:
tf_validation_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [ ]:
text = "Hello, my dog is not cute"
tokenizer(text).to_tf_dataset()

AttributeError: ignored

In [ ]:
text = "Hello, my dog is not cute"
model.predict(tokenizer(text,padding=True))

ValueError: ignored

In [ ]:
model(**tokenizer(text,padding=True,return_tensors='tf'))

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 3.5198414, -3.727813 ]], dtype=float32)>, hidden_states=None, attentions=None)

In [ ]:
text = "Hello, my dog is not cute"
logits = model(**tokenizer(text,padding=True,return_tensors='tf')).logits
class_pred = np.argmax(logits,axis=1)
class_pred

array([0])

In [ ]:
text = "Hello, my dog is cute"
logits = model(**tokenizer(text,padding=True,return_tensors='tf')).logits
class_pred = np.argmax(logits,axis=1)
class_pred

array([1])

In [ ]:
def text_predict(text):
  logits = model(**tokenizer(text,padding=True,return_tensors='tf')).logits
  class_pred = np.argmax(logits,axis=1)
  print(class_pred)
  if class_pred[0] == 0:
    return 'Negative'
  else:
    return 'Positive'

In [ ]:
text_predict("are you ok?")

[1]


'Positive'

In [ ]:
text_predict('i am good')

[1]


'Positive'

In [ ]:
text_predict('i am not good')

[0]


'Negative'

In [ ]:
text_predict('i am not bad')

[1]


'Positive'

In [ ]:
text_predict('i am bad')

[0]


'Negative'

In [ ]:
text_predict('this is nice')

[1]


'Positive'

In [ ]:
model.save('distilbert-base-uncased-sst2-3epochs')

In [ ]:
import tensorflow as tf
model2 = tf.saved_model.load('distilbert-base-uncased-sst2-3epochs')

In [ ]:
text = "Hello, my dog is cute"
logits = model2(**tokenizer(text,padding=True,return_tensors='tf')).logits
class_pred = np.argmax(logits,axis=1)
class_pred

ValueError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
